In [ ]:
##### 현재 최종본
##### 진짜 최종본
import paramiko
import os
import time
ISCSI_SERVER_IP   = "10.4.72.3"   # iSCSI Target 서버
ISCSI_CLIENT_IP   = "10.4.80.11"  # iSCSI Initiator + Samba 서버
SAMBA_CLIENT_IP   = "10.4.80.12"   # Samba 클라이언트 + 운영 서버(웹/FTP/DB)
NFS_CLIENT_IP     = "10.4.32.100"
USER              = "root"
PW                = "asd123!@"
TARGET_IQN   = "iqn.2025-10.com.total1:storage.target1"
BLOCK_DEVICE = "/dev/sda"           
IQN_FILE     = "/jupyter/pro/iqn.txt"
MOUNT_POINT  = "/mnt/logs"
DOMAIN       = "team8"               
SMB_USER_UB  = "ub"
SMB_USER_WIN = "win"
SMB_PASS     = "asd123!@"
LOG_DIR = "/mnt/logs"
LOG_SSH = f"{LOG_DIR}/ssh_log.txt"
LOG_JUPYTER = f"{LOG_DIR}/jupyter_log.txt"
FTP_SSH = f"{LOG_DIR}/ftp_log.txt"
DNS_DOMAIN  = "team8.com"
LOCAL_IP    = "172.16.254.10"
DNS_FILE    = f"/etc/bind/db.{DNS_DOMAIN}"
DNS_DATA    = "/jupyter/dns/dns.txt"
SERVERS = {
    "ISCSI_SERVER": {
        "ip": "10.4.72.3",
        "user": "root",
        "pw": "asd123!@"
    },
    "STORAGE_SERVER": {   # iSCSI 클라이언트 + SAMBA + NFS
        "ip": "10.4.80.11",
        "user": "root",
        "pw": "asd123!@"
    },
    "WEB_SERVER": {       # SAMBA CLIENT + 운영 서버
        "ip": "10.4.80.12",
        "user": "root",
        "pw": "asd123!@"
    },
    "NFS_CLIENT": {       # NFS 클라이언트
        "ip": "10.4.32.100",
        "user": "root",
        "pw": "asd123!@"
    }
}

from netmiko import ConnectHandler
DEVICE_INFO = {
    "R1": {"ip": "10.4.0.1", "type": "cisco_ios", "username": "R1", "password": "cisco"},
    "R2": {"ip": "10.4.32.1", "type": "cisco_ios", "username": "R2", "password": "cisco"},
    "R3": {"ip": "10.4.64.1", "type": "cisco_ios", "username": "R3", "password": "cisco"},
    "R4": {"ip": "192.168.100.1", "type": "cisco_ios", "username": "R4", "password": "cisco"},
    "R5": {"ip": "10.4.128.1", "type": "cisco_ios", "username": "R5", "password": "cisco"},
    "R6": {"ip": "10.4.72.1", "type": "cisco_ios", "username": "R6", "password": "cisco"},
    "SW1": {"ip": "10.4.32.2", "type": "cisco_ios", "username": "S1", "password": "cisco"},
    "SW2": {"ip": "10.4.72.2", "type": "cisco_ios", "username": "S2", "password": "cisco"},
    "SW3": {"ip": "192.168.100.2", "type": "cisco_ios", "username": "S3", "password": "cisco"},
    "SW4": {"ip": "10.4.160.2", "type": "cisco_ios", "username": "S4", "password": "cisco"},
    "SW5": {"ip": "10.4.160.3", "type": "cisco_ios", "username": "S5", "password": "cisco"},
    "SW6": {"ip": "10.4.160.4", "type": "cisco_ios", "username": "S6", "password": "cisco"},
    "SW7": {"ip": "10.4.128.2", "type": "cisco_ios", "username": "S7", "password": "cisco"},
    "SW8": {"ip": "10.4.128.3", "type": "cisco_ios", "username": "S8", "password": "cisco"},
    "SW9": {"ip": "10.4.128.4", "type": "cisco_ios", "username": "S9", "password": "cisco"},
    "SW10": {"ip": "10.4.80.2", "type": "cisco_ios", "username": "S10", "password": "cisco"},
}

def cmd(ip, command): # 원격 서버에 명령 실행 후 출력 반환
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=USER, password=PW, port=22)
    stdin, stdout, stderr = cli.exec_command(command)
    out = stdout.read().decode().strip()
    cli.close()
    return out
    
def detect_os(ip):  # 원격 서버의 OS 종류 감지
    info = cmd(ip, "cat /etc/os-release || echo UNKNOWN")
    if "Rocky" in info:
        return "Rocky"
    elif "Ubuntu" in info:
        return "Ubuntu"
    return "Unknown"


def iscsi_server_setup():  # iSCSI 서버 방화벽 해제 및 패키지 설치
    os_type = detect_os(ISCSI_SERVER_IP)
    print(f"감지된 서버 OS: {os_type}")

    if os_type == "Ubuntu":
        cmd(ISCSI_SERVER_IP, "ufw disable || true")
        cmd(ISCSI_SERVER_IP, "apt update -y && apt install -y targetcli-fb expect || apt install -y targetcli expect")
    else:
        cmd(ISCSI_SERVER_IP, "setenforce 0 || true")
        cmd(ISCSI_SERVER_IP, "systemctl stop firewalld || true")
        cmd(ISCSI_SERVER_IP, "dnf -y install epel-release || dnf -y install epel-next-release")
        cmd(ISCSI_SERVER_IP, "dnf install -y targetcli expect || dnf install -y targetcli-fb expect")

    print("iSCSI 서버 및 Expect 설치 완료")


def iscsi_server_target_config():  # iSCSI 타깃 구성
    if not os.path.exists(IQN_FILE):
        print(f"클라이언트 IQN 파일 없음: {IQN_FILE}")
        return

    with open(IQN_FILE, "r", encoding="utf-8") as f:
        client_iqn = f.read().strip()
    print(f"불러온 클라이언트 IQN: {client_iqn}")

    cmds = [
        "mkdir -p /iscsi_disks && chmod 777 /iscsi_disks",
        "targetcli /backstores/block delete disk100 || true",
        f"targetcli /iscsi delete {TARGET_IQN} || true",
        f"targetcli /backstores/block create disk100 {BLOCK_DEVICE}",
        f"targetcli /iscsi create {TARGET_IQN}",
        f"targetcli /iscsi/{TARGET_IQN}/tpg1/luns create /backstores/block/disk100",
        f"targetcli /iscsi/{TARGET_IQN}/tpg1/acls create {client_iqn}",
        f"targetcli /iscsi/{TARGET_IQN}/tpg1/portals create 0.0.0.0",
        "targetcli saveconfig"
    ]
    for c in cmds:
        cmd(ISCSI_SERVER_IP, c)

    print("iSCSI 타깃 생성 완료")


def iscsi_server_restart():  # iSCSI 서비스 및 포트 확인
    cmd(ISCSI_SERVER_IP, "systemctl enable target || systemctl enable rtslib-fb-targetctl || true")
    cmd(ISCSI_SERVER_IP, "systemctl restart target || systemctl restart rtslib-fb-targetctl || true")
    cmd(ISCSI_SERVER_IP, "setenforce 0 || true")
    cmd(ISCSI_SERVER_IP, "systemctl stop firewalld || true")
    port_check = cmd(ISCSI_SERVER_IP, "ss -tuln | grep 3260 || echo 'Port 3260 not open'")
    print(port_check)
    print("iSCSI 서버 재시작 및 포트 확인 완료")


def iscsi_client_install():  # iSCSI 클라이언트 설치 및 IQN 파일 생성
    os_type = detect_os(ISCSI_CLIENT_IP)
    print(f"감지된 클라이언트 OS: {os_type}")
    if os_type == "Ubuntu":
        cmd(ISCSI_CLIENT_IP, "apt update -y && apt install -y open-iscsi")
        cmd(ISCSI_CLIENT_IP, "systemctl enable iscsid && systemctl start iscsid")
        cmd(ISCSI_CLIENT_IP, "ufw disable || true")
    else:
        cmd(ISCSI_CLIENT_IP, "dnf install -y iscsi-initiator-utils iscsi-initiator-utils-iscsiuio")
        cmd(ISCSI_CLIENT_IP, "systemctl enable iscsid && systemctl start iscsid")
        cmd(ISCSI_CLIENT_IP, "systemctl enable iscsi && systemctl start iscsi")
        cmd(ISCSI_CLIENT_IP, "systemctl stop firewalld || true")

    print("iSCSI 클라이언트 설치 및 IQN 생성 완료\n")

def iscsi_client_iqn():  
    print("잠시 대기")
    time.sleep(10)
    print("대기 끝")
    exist_check = cmd(ISCSI_CLIENT_IP, "test -f /etc/iscsi/initiatorname.iscsi && echo EXIST || echo MISSING")
    if "EXIST" in exist_check:
        iqn = cmd(ISCSI_CLIENT_IP, "grep InitiatorName /etc/iscsi/initiatorname.iscsi | cut -d= -f2").strip()
        if iqn:
            print(f"기존 IQN 감지됨: {iqn}")
        else:
            print("기존 파일은 있지만 IQN이 비어 있습니다. 재생성합니다.")
            iqn = cmd(ISCSI_CLIENT_IP, "/usr/sbin/iscsi-iname").strip()
            cmd(ISCSI_CLIENT_IP, f"echo 'InitiatorName={iqn}' > /etc/iscsi/initiatorname.iscsi")
    else:
        print("IQN 파일이 존재하지 않아 새로 생성합니다.")
        iqn = cmd(ISCSI_CLIENT_IP, "/usr/sbin/iscsi-iname").strip()
        cmd(ISCSI_CLIENT_IP, f"mkdir -p /etc/iscsi && echo 'InitiatorName={iqn}' > /etc/iscsi/initiatorname.iscsi")

    if not iqn:
        print("IQN 생성 실패 — 수동 확인 필요 (/usr/sbin/iscsi-iname 실행 결과 확인)")
        return None

    with open(IQN_FILE, "w", encoding="utf-8") as out:
        out.write(iqn)

    print(f"IQN 생성/저장 완료 → {IQN_FILE}")
    print(f"   IQN: {iqn}")
    return iqn

def iscsi_client_login():
    cmd(ISCSI_CLIENT_IP, "iscsiadm -m node -u || true")
    cmd(ISCSI_CLIENT_IP, f"iscsiadm -m discovery -t sendtargets -p {ISCSI_SERVER_IP}")
    cmd(ISCSI_CLIENT_IP, f"iscsiadm -m node -T {TARGET_IQN} -p {ISCSI_SERVER_IP} --login")
    cmd(ISCSI_CLIENT_IP, "sleep 2; iscsiadm -m session --rescan; udevadm trigger --subsystem-match=scsi --action=add")
    print("iSCSI 로그인 및 디스크 재탐색 완료")

def iscsi_client_mount():
    print("iSCSI 디스크 파티션/마운트")
    cmd(ISCSI_CLIENT_IP, "iscsiadm -m session --rescan || true")
    cmd(ISCSI_CLIENT_IP, "udevadm trigger --subsystem-match=scsi --action=add || true")
    time.sleep(3)
    bypath = cmd(ISCSI_CLIENT_IP, "ls /dev/disk/by-path | grep -i iscsi | grep 'lun-0' | head -n 1").strip()
    if not bypath:
        print("iSCSI 디스크 탐지 실패")
        return
    dev = cmd(ISCSI_CLIENT_IP, f"readlink -f /dev/disk/by-path/{bypath}").strip()
    part = f"{dev}1"

    part_exist = cmd(ISCSI_CLIENT_IP, f"ls {part} 2>/dev/null || echo NONE").strip()
    if "NONE" in part_exist:
        print(cmd(ISCSI_CLIENT_IP, f"parted -s {dev} mklabel gpt mkpart primary ext4 0% 100%"))
        cmd(ISCSI_CLIENT_IP, "partprobe; udevadm settle; sleep 2")

    fs_check = cmd(ISCSI_CLIENT_IP, f"blkid {part} | grep ext4 || echo NONE").strip()
    if "NONE" in fs_check:
        print(cmd(ISCSI_CLIENT_IP, f"mkfs.ext4 -F {part}"))
    cmd(ISCSI_CLIENT_IP, f"mkdir -p {MOUNT_POINT}")
    cmd(ISCSI_CLIENT_IP, f"mount {part} {MOUNT_POINT}")
    print(cmd(ISCSI_CLIENT_IP, f"df -h | grep {MOUNT_POINT} || echo '마운트 실패'"))

def quota(): # 쿼터 활성화 및 사용자별 용량 제한 설정
    os_info = detect_os(ISCSI_CLIENT_IP)
    if "Rocky" in os_info:
        print(cmd(ISCSI_CLIENT_IP, "dnf install -y quota"))
    elif "Ubuntu" in os_info:
        print(cmd(ISCSI_CLIENT_IP, "apt-get update && apt-get install -y quota"))

    fstab_check = cmd(ISCSI_CLIENT_IP, f"grep '{MOUNT_POINT}' /etc/fstab || echo NONE")
    if "NONE" in fstab_check:
        print(cmd(ISCSI_CLIENT_IP, f"echo '/dev/sda1 {MOUNT_POINT} ext4 defaults,usrquota 0 0' >> /etc/fstab"))
    else:
        print(cmd(ISCSI_CLIENT_IP, f"sed -i '/{MOUNT_POINT}/ s/defaults/defaults,usrquota/' /etc/fstab || true"))
    print(cmd(ISCSI_CLIENT_IP, f"mount -o remount {MOUNT_POINT} || mount -a"))

    print(cmd(ISCSI_CLIENT_IP, f"quotaoff {MOUNT_POINT} 2>/dev/null || true"))
    print(cmd(ISCSI_CLIENT_IP, f"rm -f {MOUNT_POINT}/aquota.user {MOUNT_POINT}/aquota.group || true"))
    print(cmd(ISCSI_CLIENT_IP, f"quotacheck -cum {MOUNT_POINT}"))
    print(cmd(ISCSI_CLIENT_IP, f"quotaon {MOUNT_POINT}"))

    print(cmd(ISCSI_CLIENT_IP, f"useradd -m -U -d {MOUNT_POINT}/{SMB_USER_UB} {SMB_USER_UB} 2>/dev/null || true"))
    print(cmd(ISCSI_CLIENT_IP, f"useradd -m -U -d {MOUNT_POINT}/{SMB_USER_WIN} {SMB_USER_WIN} 2>/dev/null || true"))

    UB_BLOCKS  = 15 * 1024 * 1024
    WIN_BLOCKS = 5  * 1024 * 1024
    print(cmd(ISCSI_CLIENT_IP, f"setquota -u {SMB_USER_UB}  0 {UB_BLOCKS}  0 0 {MOUNT_POINT}"))
    print(cmd(ISCSI_CLIENT_IP, f"setquota -u {SMB_USER_WIN} 0 {WIN_BLOCKS} 0 0 {MOUNT_POINT}"))

    print(cmd(ISCSI_CLIENT_IP, f"mkdir -p {MOUNT_POINT}/{SMB_USER_UB} {MOUNT_POINT}/{SMB_USER_WIN}"))
    print(cmd(ISCSI_CLIENT_IP, f"chown -R {SMB_USER_UB}:{SMB_USER_UB} {MOUNT_POINT}/{SMB_USER_UB}"))
    print(cmd(ISCSI_CLIENT_IP, f"chown -R {SMB_USER_WIN}:{SMB_USER_WIN} {MOUNT_POINT}/{SMB_USER_WIN}"))
    print("쿼터 설정/권한 완료\n")
    print(cmd(ISCSI_CLIENT_IP, "repquota -u -a || echo '쿼터 정보 없음'"))

def samba_server_setup():
    os_type = detect_os(ISCSI_CLIENT_IP)
    print(f"OS: {os_type}")

    if os_type == "Rocky":
        cmd(ISCSI_CLIENT_IP, "dnf install -y samba samba-client cifs-utils")
        cmd(ISCSI_CLIENT_IP, "systemctl stop firewalld; systemctl disable firewalld")
        cmd(ISCSI_CLIENT_IP, "setenforce 0 || true")
        cmd(ISCSI_CLIENT_IP, "systemctl enable --now smb nmb")
    elif os_type == "Ubuntu":
        cmd(ISCSI_CLIENT_IP, "apt update -y && apt install -y samba samba-client cifs-utils")
        cmd(ISCSI_CLIENT_IP, "ufw disable || true")
        cmd(ISCSI_CLIENT_IP, "systemctl enable --now smbd nmbd || systemctl enable --now smb nmb")

    cmd(ISCSI_CLIENT_IP, f"mkdir -p {MOUNT_POINT}/{SMB_USER_UB} {MOUNT_POINT}/{SMB_USER_WIN}")
    cmd(ISCSI_CLIENT_IP, f"usermod -d {MOUNT_POINT}/{SMB_USER_UB} {SMB_USER_UB} || true")
    cmd(ISCSI_CLIENT_IP, f"usermod -d {MOUNT_POINT}/{SMB_USER_WIN} {SMB_USER_WIN} || true")
    cmd(ISCSI_CLIENT_IP, f"chown -R {SMB_USER_UB}:{SMB_USER_UB} {MOUNT_POINT}/{SMB_USER_UB}")
    cmd(ISCSI_CLIENT_IP, f"chown -R {SMB_USER_WIN}:{SMB_USER_WIN} {MOUNT_POINT}/{SMB_USER_WIN}")

    cmd(ISCSI_CLIENT_IP, f"(echo '{SMB_PASS}'; echo '{SMB_PASS}') | smbpasswd -a {SMB_USER_UB}")
    cmd(ISCSI_CLIENT_IP, f"(echo '{SMB_PASS}'; echo '{SMB_PASS}') | smbpasswd -a {SMB_USER_WIN}")

    cmd(ISCSI_CLIENT_IP, "testparm -s || true")

    if os_type == "Ubuntu":
        cmd(ISCSI_CLIENT_IP, "systemctl restart smbd nmbd || systemctl restart smb nmb")
    else:
        cmd(ISCSI_CLIENT_IP, "systemctl restart smb nmb")

    print("Samba 서버(homes) 기본 구성 완료\n")

def samba_mount(): # samba mount
    os_type = detect_os(SAMBA_CLIENT_IP)
    if os_type == "Rocky":
        print(cmd(SAMBA_CLIENT_IP, "dnf install -y samba-client cifs-utils"))
        print(cmd(SAMBA_CLIENT_IP, "systemctl stop firewalld; systemctl disable firewalld"))
    elif os_type == "Ubuntu":
        print(cmd(SAMBA_CLIENT_IP, "apt update -y && apt install -y samba-client cifs-utils"))
        print(cmd(SAMBA_CLIENT_IP, "ufw disable || true"))

    print(cmd(SAMBA_CLIENT_IP, f"mkdir -p {MOUNT_POINT}; chmod 777 {MOUNT_POINT}"))

    fstab_entry = f"//{ISCSI_CLIENT_IP}/homes {MOUNT_POINT} cifs username={SMB_USER_UB},password={SMB_PASS},rw,vers=3.0 0 0"
    chk = cmd(SAMBA_CLIENT_IP, f"grep '{MOUNT_POINT}' /etc/fstab || echo NONE")
    if "NONE" in chk:
        print(cmd(SAMBA_CLIENT_IP, f"echo '{fstab_entry}' >> /etc/fstab"))
    print(cmd(SAMBA_CLIENT_IP, f"umount -f {MOUNT_POINT} || true"))
    out = cmd(SAMBA_CLIENT_IP, "mount -a; sleep 1; df -h | grep /mnt/logs || echo FAIL")
    print(out)
    if "FAIL" in out:
        print("Samba 마운트 실패 (네트워크/서비스/계정 확인)")
    else:
        for f in ["ssh_log.txt", "ftp_log.txt", "jupyter_log.txt"]:
            print(cmd(SAMBA_CLIENT_IP, f"touch {MOUNT_POINT}/{f}; chmod 666 {MOUNT_POINT}/{f}"))
        print("Samba homes 마운트 완료\n")

def nfs_server_setup():
    print(f"NFS 서버 설정 중 ({ISCSI_CLIENT_IP})")
    os_type = detect_os(ISCSI_CLIENT_IP)

    if os_type == "Ubuntu":
        cmds = [
            "apt update -y && apt install -y nfs-kernel-server",
            "systemctl enable --now nfs-kernel-server",
            f"mkdir -p {MOUNT_POINT}/{SMB_USER_UB}",
            f"chmod 755 {MOUNT_POINT}/{SMB_USER_UB}",
            f"echo '{MOUNT_POINT}/{SMB_USER_UB} 172.16.0.11/16(ro,sync)' > /etc/exports",
            "exportfs -r",
            "systemctl restart nfs-kernel-server",
            "showmount -e"
        ]
    else:  
        cmds = [
            "dnf install -y nfs-utils",
            "systemctl enable --now nfs-server",
            f"mkdir -p {MOUNT_POINT}/{SMB_USER_UB}",
            f"chmod 755 {MOUNT_POINT}/{SMB_USER_UB}",
            f"echo '{MOUNT_POINT}/{SMB_USER_UB} 172.16.0.11/16(ro,sync)' > /etc/exports",
            "exportfs -r",
            "systemctl restart nfs-server",
            "showmount -e"
        ]
    for c in cmds:
        print(cmd(ISCSI_CLIENT_IP, c))
    print(f"NFS 서버 설정 완료 ({MOUNT_POINT}/ub → 172.16.0.11/16 ro 공유)\n")

def tools_ufw():
    os_type = detect_os(SAMBA_CLIENT_IP)
    if os_type == "Ubuntu":
        print(cmd(SAMBA_CLIENT_IP, "apt update -y"))
        print(cmd(SAMBA_CLIENT_IP, "apt install -y net-tools vim git curl gnupg lsb-release ca-certificates apt-transport-https"))
        print(cmd(SAMBA_CLIENT_IP, "ufw disable || true"))
    elif os_type == "Rocky":
        print(cmd(SAMBA_CLIENT_IP, "dnf install -y epel-release"))
        print(cmd(SAMBA_CLIENT_IP, "dnf install -y net-tools vim git curl firewalld"))
        print(cmd(SAMBA_CLIENT_IP, "systemctl stop firewalld; systemctl disable firewalld"))
    print("운영서버 기본도구/방화벽 설정 완료\n")

def user_create():
    u1 = input("첫 번째 사용자 이름 입력: ").strip()
    u2 = input("두 번째 사용자 이름 입력: ").strip()
    for usr in [u1, u2]:
        chk = cmd(SAMBA_CLIENT_IP, f"id -u {usr} > /dev/null 2>&1 || echo notfound")
        if "notfound" in chk:
            print(cmd(SAMBA_CLIENT_IP, f"useradd -m {usr}"))
            print(cmd(SAMBA_CLIENT_IP, f"chmod 777 /home/{usr}"))
            print(f"새로 생성된 사용자: {usr}")
        else:
            print(f"이미 존재하는 사용자: {usr}")
    global WEB_USER1, WEB_USER2
    WEB_USER1, WEB_USER2 = u1, u2

def apache():
    os_type = detect_os(SAMBA_CLIENT_IP)
    if os_type == "Ubuntu":
        print(cmd(SAMBA_CLIENT_IP, "apt install -y apache2"))
        print(cmd(SAMBA_CLIENT_IP, "systemctl enable apache2 && systemctl start apache2"))
    else:
        print(cmd(SAMBA_CLIENT_IP, "dnf install -y httpd"))
        print(cmd(SAMBA_CLIENT_IP, "systemctl enable httpd && systemctl start httpd"))
    print("Apache 설치 완료\n")
    
def virtualhost():
    global WEB_USER1, WEB_USER2
    if 'WEB_USER1' not in globals() or 'WEB_USER2' not in globals():
        user_create()

    vhost_conf = f"""
<VirtualHost *:80>
    ServerName ww1.{DOMAIN}.com
    DocumentRoot /home/{WEB_USER1}
    ErrorLog /var/log/apache2/ww1_error.log
    CustomLog /var/log/apache2/ww1_access_log combined
    <Directory /home/{WEB_USER1}/>
        AllowOverride None
        Options Indexes FollowSymLinks
        Require all granted
    </Directory>
</VirtualHost>

<VirtualHost *:80>
    ServerName ww2.{DOMAIN}.com
    DocumentRoot /home/{WEB_USER2}
    ErrorLog /var/log/apache2/ww2_error.log
    CustomLog /var/log/apache2/ww2_access_log combined
    <Directory /home/{WEB_USER2}/>
        AllowOverride None
        Options Indexes FollowSymLinks
        Require all granted
    </Directory>
</VirtualHost>
"""
    os_type = detect_os(SAMBA_CLIENT_IP)
    remote_path = "/etc/httpd/conf.d/vhost.conf" if os_type == "Rocky" else "/etc/apache2/sites-available/vhost.conf"
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(SAMBA_CLIENT_IP, username=USER, password=PW)
    sftp = cli.open_sftp()
    with sftp.file(remote_path, "w") as f:
        f.write(vhost_conf)
    sftp.close()
    cli.close()

    if os_type == "Ubuntu":
        print(cmd(SAMBA_CLIENT_IP, "a2ensite vhost.conf && a2dissite 000-default.conf && systemctl reload apache2"))
    else:
        print(cmd(SAMBA_CLIENT_IP, "systemctl restart httpd"))
    print("가상호스트 설정 완료\n")

def ftp():
    os_type = detect_os(SAMBA_CLIENT_IP)
    if os_type == "Ubuntu":
        print(cmd(SAMBA_CLIENT_IP, "apt install -y vsftpd"))
    else:
        print(cmd(SAMBA_CLIENT_IP, "dnf install -y vsftpd"))
    ftp_conf = """
listen_ipv6=YES
anonymous_enable=YES
local_enable=YES
write_enable=YES
chroot_local_user=YES
allow_writeable_chroot=YES
"""
    remote_conf = "/etc/vsftpd.conf"
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(SAMBA_CLIENT_IP, username=USER, password=PW)
    sftp = cli.open_sftp()
    with sftp.file(remote_conf, "w") as f:
        f.write(ftp_conf)
    sftp.close()
    cli.close()
    print(cmd(SAMBA_CLIENT_IP, "systemctl restart vsftpd && systemctl enable vsftpd"))
    print("FTP 설정 완료\n")

def php():
    os_type = detect_os(SAMBA_CLIENT_IP)
    if os_type == "Ubuntu":
        print(cmd(SAMBA_CLIENT_IP, "apt install -y php libapache2-mod-php php-mysql php-cli php-curl php-gd php-mbstring php-xml php-zip"))
        print(cmd(SAMBA_CLIENT_IP, "a2enmod php* && systemctl restart apache2"))
    else:
        print(cmd(SAMBA_CLIENT_IP, "dnf install -y php php-mysqlnd php-cli php-gd php-mbstring php-xml php-zip"))
        print(cmd(SAMBA_CLIENT_IP, "systemctl restart httpd"))

    if 'WEB_USER1' not in globals():
        user_create_ops()
    for i in range(1, 5):
        php_file = f"/home/{WEB_USER1}/{i}.php"
        content = f"<?php echo '<h1>{i}번 페이지</h1>'; ?>"
        cli = paramiko.SSHClient()
        cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        cli.connect(SAMBA_CLIENT_IP, username=USER, password=PW)
        sftp = cli.open_sftp()
        with sftp.file(php_file, "w") as f:
            f.write(content)
        sftp.close()
        cli.close()
        print(cmd(SAMBA_CLIENT_IP, f"chmod 644 {php_file}"))
    print("PHP 테스트 페이지 생성 완료\n")

def mariadb():
    os_type = detect_os(SAMBA_CLIENT_IP)
    if os_type == "Ubuntu":
        print(cmd(SAMBA_CLIENT_IP, "apt install -y mariadb-server"))
    else:
        print(cmd(SAMBA_CLIENT_IP, "dnf install -y mariadb-server"))
    print(cmd(SAMBA_CLIENT_IP, "systemctl enable mariadb && systemctl start mariadb"))

    setup_sql = """
    ALTER USER 'root'@'localhost' IDENTIFIED BY 'asd123!@';
    DELETE FROM mysql.user WHERE User='';
    DROP DATABASE IF EXISTS test;
    DELETE FROM mysql.db WHERE Db='test' OR Db='test\\_%';
    FLUSH PRIVILEGES;
    """
    print(cmd(SAMBA_CLIENT_IP, f"mysql -uroot -e \"{setup_sql}\""))
    create_sql = """
    CREATE USER IF NOT EXISTS 'team8'@'%' IDENTIFIED BY 'asd123!@';
    CREATE DATABASE IF NOT EXISTS team8;
    GRANT ALL PRIVILEGES ON team8.* TO 'team8'@'%';
    FLUSH PRIVILEGES;
    """
    print(cmd(SAMBA_CLIENT_IP, f"mysql -uroot -pasd123!@ -e \"{create_sql}\""))
    print("MariaDB 설정 완료\n")

def phpmyadmin():
    os_type = detect_os(SAMBA_CLIENT_IP)
    if os_type == "Ubuntu":
        preconf = """
echo 'phpmyadmin phpmyadmin/dbconfig-install boolean true' | debconf-set-selections
echo 'phpmyadmin phpmyadmin/app-password-confirm password asd123!@' | debconf-set-selections
echo 'phpmyadmin phpmyadmin/mysql/admin-pass password asd123!@' | debconf-set-selections
echo 'phpmyadmin phpmyadmin/mysql/app-pass password asd123!@' | debconf-set-selections
echo 'phpmyadmin phpmyadmin/reconfigure-webserver multiselect apache2' | debconf-set-selections
"""
        print(cmd(SAMBA_CLIENT_IP, preconf))
        print(cmd(SAMBA_CLIENT_IP, "DEBIAN_FRONTEND=noninteractive apt install -y phpmyadmin"))
        conf_content = """Alias /phpmyadmin /usr/share/phpmyadmin
<Directory /usr/share/phpmyadmin>
    Options SymLinksIfOwnerMatch
    DirectoryIndex index.php
</Directory>
"""
        remote_conf = "/etc/apache2/conf-available/phpmyadmin.conf"
        cli = paramiko.SSHClient()
        cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        cli.connect(SAMBA_CLIENT_IP, username=USER, password=PW)
        sftp = cli.open_sftp()
        with sftp.file(remote_conf, "w") as f:
            f.write(conf_content)
        sftp.close()
        cli.close()
        print(cmd(SAMBA_CLIENT_IP, "a2enconf phpmyadmin && systemctl reload apache2"))
    else:
        print(cmd(SAMBA_CLIENT_IP, "dnf install -y epel-release"))
        print(cmd(SAMBA_CLIENT_IP, "dnf install -y phpMyAdmin"))
        alias_conf = """
Alias /phpmyadmin /usr/share/phpMyAdmin
<Directory /usr/share/phpMyAdmin/>
    AddDefaultCharset UTF-8
    <IfModule mod_authz_core.c>
        <RequireAll>
            Require all granted
        </RequireAll>
    </IfModule>
</Directory>
"""
        remote_path = "/etc/httpd/conf.d/phpMyAdmin.conf"
        cli = paramiko.SSHClient()
        cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        cli.connect(SAMBA_CLIENT_IP, username=USER, password=PW)
        sftp = cli.open_sftp()
        with sftp.file(remote_path, "w") as f:
            f.write(alias_conf)
        sftp.close()
        cli.close()
        print(cmd(SAMBA_CLIENT_IP, "systemctl restart httpd"))
    print("phpMyAdmin 설치/설정 완료\n")
#로그전용
log_cli = paramiko.SSHClient()
log_cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
log_cli.connect(SAMBA_CLIENT_IP, username=USER, password=PW, port=22)

def ssh_log():
    stdin, stdout, stderr = log_cli.exec_command("tail -n 300 /var/log/auth.log || tail -n 300 /var/log/secure")
    fail = 0
    for line in stdout.readlines():
        line = line.strip()
        if not line:
            continue
        time = line[0:19]
        ip = "unknown"
        if "root from" in line:
            a = line.find("root from")
            b = line.find("port")
            ip = line[a+10:b].strip()
        if "Accepted password" in line:
            fail = 0
            log_txt = f"SSH접속 허용 {time} → {ip}"
        elif "Failed password" in line:
            fail += 1
            log_txt = f"SSH접속 실패 {time} → {ip}"
            if fail >= 5:
                log_txt += " [IP 차단 필요]"
                fail = 0
        else:
            continue
        print(log_txt)
        log_cli.exec_command(f"echo '{log_txt}' >> {LOG_SSH}")

def jupyter_log():
    stdin, stdout, stderr = log_cli.exec_command("tail -n 100 /var/log/jupyter.log")
    fail = 0
    for line in stdout.readlines():
        line = line.strip()
        if not line:
            time.sleep(0.2)
            continue

        time = line[3:26]
        a = line.find("@")
        b = line.find(")")
        ip = line[a+1:b]
        if "302 POST" in line:
            fail = 0
            log_txt = f"로그인 성공! TIME: {time} → IP: {ip}"
            print(log_txt)
            with open(LOG_JUPYTER, "a", encoding="utf-8") as out:
                out.write(log_txt + "\n")
        elif "401 POST" in line:
            fail += 1
            log_txt = f"로그인 실패! TIME: {time} → IP: {ip}"
            if fail >= 5:
                log_txt += " [IP 차단 필요]"
                fail = 0
        else:
            continue
        print(log_txt)
        log_cli.exec_command(f"echo '{log_txt}' >> {LOG_JUPYTER}")

def ftp_log():
    stdin, stdout, stderr = log_cli.exec_command("tail -n 100 /var/log/vsftpd.log")
    for line in stdout.readlines():
        line = line.strip()
        if not line:
            time.sleep(0.2)
            continue
        a = line.find("::ffff:")
        b = line.find('"', a)
        ip = line[a+7:b]
        time = line[0:19]
        if "CONNECT" in line:
            log_txt = f"로그인 시도 {time} → {ip}"
        elif "FAIL LOGIN" in line:
            log_txt = f"로그인 실패 {time} → {ip}"
        elif "OK LOGIN" in line:
            log_txt = f"로그인 성공 {time} → {ip}"
        elif "OK UPLOAD" in line:
            log_txt = f"업로드 성공 {time} → {ip}"
        elif "OK DOWNLOAD" in line:
            log_txt = f"다운로드 성공 {time} → {ip}"
        elif "OK DELETE" in line:
            log_txt = f"삭제 성공 {time} → {ip}"
        else:
            continue

        print(log_txt)
        log_cli.exec_command(f"echo '{log_txt}' >> {LOG_DIR}/ftp_log.txt")

def nfs_client_mount():
    os_type = detect_os(NFS_CLIENT_IP)
    print(f"NFS 클라이언트 설정 중 ({NFS_CLIENT_IP})")
    
    if os_type == "Ubuntu":
        cmds = [
            "apt update -y && apt install -y nfs-common",
            "mkdir -p /mnt/nfs_ub",
            "mount -t nfs 10.4.80.11:/mnt/logs/ub /mnt/nfs_ub || echo '마운트 실패'",
            "df -h | grep nfs_ub || echo '마운트 안됨'"
        ]
    else:  
        cmds = [
            "dnf install -y nfs-utils",
            "systemctl enable --now nfs-client.target || true",
            "mkdir -p /mnt/nfs_ub",
            "mount -t nfs 10.4.80.11:/mnt/logs/ub /mnt/nfs_ub || echo '마운트 실패'",
            "df -h | grep nfs_ub || echo '마운트 안됨'"
        ]
    for c in cmds:
        print(cmd(NFS_CLIENT_IP, c))
    print("NFS 마운트 완료 (/mnt/nfs_ub)\n")


def read_dns():
    file_path = "/jupyter/dns/" + input("파일 이름 입력 (예: dns.txt): ")
    do = []
    server = input("서버 이름 입력 (예: rk / wu / win22 / win10 / R / SW): ")
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('_')
            if parts[0] == server:
                print(f"서버: {parts[0]} 도메인주소: {parts[1]} IP: {parts[4]}")
                do.append((parts[1], parts[4]))
    return do

def clear_zone():
    with open(DNS_FILE, 'w', encoding='utf-8') as f:
        f.write(f"""$TTL    604800
@       IN      SOA     {DOMAIN}. root.{DOMAIN}. (
                              2         ; Serial
                         604800         ; Refresh
                          86400         ; Retry
                        2419200         ; Expire
                         604800 )       ; Negative Cache TTL
;
@       IN      NS      {DOMAIN}.
@       IN      A       {LOCAL_IP}
ns1     IN      A       {LOCAL_IP}

""")
    print("DNS 존 파일 초기화 완료")


def add_dns():
    g_doip = read_dns()
    with open(DNS_FILE, 'a', encoding='utf-8') as f:
        for domain, ip in g_doip:
            f.write(f"{domain}     IN      A       {ip}\n")
    print("새 DNS 레코드 추가 완료")
    print_dns()

def print_dns():
    if not os.path.exists(DNS_FILE):
        return
    with open(DNS_FILE, 'r', encoding='utf-8') as f:
        print(f.read())

def dns_reset():
    with open(DNS_DATA, 'w', encoding='utf-8') as f:
        f.write("")
    if os.path.exists(DNS_FILE):
        with open(DNS_FILE, 'r', encoding='utf-8') as f:
            lines = f.readlines()[:11]
        with open(DNS_FILE, 'w', encoding='utf-8') as f:
            f.writelines(lines)

def dns_input():
    keys = ["서버", "도메인주소", "id", "pw", "ip"]
    info = {}
    for key in keys:
        info[key] = input(f"{key}를 입력해주세요: ")
    return info

def dns_save(data):
    line = f"{data['서버']}_{data['도메인주소']}_{data['id']}_{data['pw']}_{data['ip']}"
    with open(DNS_DATA, 'a', encoding='utf-8') as f:
        f.write(line + "\n")



def select_device():
    print("접속할 장비 이름을 선택하세요:")
    print(f"사용 가능한 장비: {list(DEVICE_INFO.keys())}")
    
    choice = input("장비 이름 입력 (예: R1, SW1): ").strip().upper()

    if choice in DEVICE_INFO:
        info = DEVICE_INFO[choice].copy()  # 원본 보호 위해 복사
        info["name"] = choice

        # 사용자에게 아이디, 비밀번호 입력받기 (빈칸 시 기본값 유지)
        user = input("아이디를 입력하세요 : ".format(info["username"])).strip()
        pw = input("비밀번호를 입력하세요 : ".format(info["password"])).strip()

        if user:
            info["username"] = user
        if pw:
            info["password"] = pw

        return info
    else:
        print("❌ 잘못된 장비 이름입니다.")
        return None

# === 3️⃣ 명령 선택 ===
def select_command():
    commands = {
        1: "show running-config",
        2: "show ip interface brief",
        3: "show vtp status",
        4: "show vlan-switch brief",
        5: "show spanning-tree brief",
        6: "show mac address-table",
        7: "show frame-relay map",
        8: "show version",
        9: "show interfaces trunk",
        10: "show ip route",
        11: "ping 8.8.8.8",
    }

    print("==== Cisco Command Menu ====")
    for key, val in commands.items():
        print(f"{key}. {val}")
    print("=============================")

    try:
        choice = int(input("실행할 명령 번호를 입력하세요: "))
        if choice in commands:
            return commands[choice]
        else:
            print("❌ 잘못된 번호입니다.")
            return None
    except ValueError:
        print("❌ 숫자를 입력하세요.")
        return None

# === 4️⃣ 장비 접속 (SSH) ===
def connect_device(device_info):
    device = {
        "device_type": device_info["type"],
        "host": device_info["ip"],
        "username": device_info["username"],
        "password": device_info["password"],
        "secret": "cisco",  # enable password
        "fast_cli": False,
    }
    net_connect = ConnectHandler(**device)
    net_connect.enable()
    return net_connect

# === 5️⃣ 설정 저장 ===
def save_running_config(output, device_name, device_ip):
    file_name = f"{device_name}_{device_ip}_running-config.txt"
    with open(file_name, "w") as f:
        f.write(output)
    print(f"💾 전체 설정 백업 저장됨: {file_name}")

# === 6️⃣ 인터페이스 목록 추출 ===
def extract_interfaces(output):
    lines = output.splitlines()
    interface_lines = [line for line in lines if line.startswith("interface")]
    return [line.split()[1] for line in interface_lines if len(line.split()) > 1]

# === 7️⃣ IP route 추출 ===
def extract_ip_routes(output):
    lines = output.splitlines()
    return [line for line in lines if line.startswith("ip route")]

# === 8️⃣ 인터페이스 설정 출력 ===
def show_interface_config(output, interface_name):
    lines = output.splitlines()
    iface_config = []
    capture = False
    for line in lines:
        if line.startswith("interface "):
            capture = (line.split()[1] == interface_name)
        elif line.startswith("interface") and capture:
            capture = False
        
        if capture or line.startswith(f"interface {interface_name}"):
            if iface_config and iface_config[-1] == line:
                continue
            iface_config.append(line)
            
    print(f"\n--------- [{interface_name} 설정] ---------")
    if iface_config:
        if not iface_config[0].startswith("interface"): 
            iface_config = [f"interface {interface_name}"] + iface_config
        
        clean_config = [line.strip() for line in iface_config if line.strip()]
        
        print("\n".join(clean_config))
    else:
        print("(설정 없음)")
    print("----------------------------------------")

# === 9️⃣ 실행 함수 ===
def run_cisco_command():
    device_info = select_device()
    if not device_info:
        return

    cmd = select_command()
    if not cmd:
        return

    print(f"\n✅ 장비: {device_info['name']} ({device_info['ip']}) 실행할 명령: {cmd}\n")
    try:
        net_connect = connect_device(device_info)
        output = net_connect.send_command(cmd)
        net_connect.disconnect()

        if "running-config" in cmd:
            save_running_config(output, device_info["name"], device_info["ip"])
            interfaces = extract_interfaces(output)
            ip_routes = extract_ip_routes(output)

            if interfaces:
                print("--------- [인터페이스 목록] ---------")
                for idx, iface in enumerate(interfaces, 1):
                    print(f"{idx}. {iface}")
                print("-------------------------------------")
                try:
                    iface_choice = int(input("확인할 인터페이스 번호를 입력하세요: "))
                    if 1 <= iface_choice <= len(interfaces):
                        show_interface_config(output, interfaces[iface_choice - 1])
                except ValueError:
                    print("❌ 잘못된 입력입니다.")
                except IndexError:
                    print("❌ 잘못된 번호입니다.")
            else:
                print("(인터페이스 설정 없음)")

            print("--------- [IP Route 설정] ---------")
            if ip_routes:
                print("\n".join(ip_routes))
            else:
                print("(설정 없음)")
            print("----------------------------------")

        else:
            print("--------- [명령 결과] ---------")
            print(output)
            print("--------------------------------")

    except Exception as e:
        print(f"❌ 연결 또는 명령 실행 실패: {e}")
def check_storage_logs():
    s = SERVERS["STORAGE_SERVER"]
    cli = ssh_connect(s["ip"], s["user"], s["pw"])
    check_print("📁 STORAGE 서버 로그 파일 점검 (/mnt/logs/ub)")

    checks = {
        "logs 디렉토리": "ls -ld /mnt/logs",
        "ssh_log.txt": "ls -l /mnt/logs/ub/ssh_log.txt 2>/dev/null",
        "ftp_log.txt": "ls -l /mnt/logs/ub/ftp_log.txt 2>/dev/null",
        "jupyter_log.txt": "ls -l /mnt/logs/ub/jupyter_log.txt 2>/dev/null",
        "파일 크기 전체": "du -h /mnt/logs/ub"
    }

    for name, cmd in checks.items():
        out = run_cmd(cli, cmd)
        print(f"▶ {name}:")
        print(out if out else "❌ 없음")
        print()

    cli.close()


def check_web_logs():
    s = SERVERS["WEB_SERVER"]
    cli = ssh_connect(s["ip"], s["user"], s["pw"])
    check_print("📁 WEB 서버 로그 확인 (/mnt/logs)")

    checks = {
        "samba 마운트 상태": "df -h | grep /mnt/logs",
        "ssh_log.txt 최근 10줄": "tail -n 10 /mnt/logs/ssh_log.txt 2>/dev/null",
        "ftp_log.txt 최근 5줄": "tail -n 5 /mnt/logs/ftp_log.txt 2>/dev/null",
        "jupyter_log.txt 최근 5줄": "tail -n 5 /mnt/logs/jupyter_log.txt 2>/dev/null"
    }

    for name, cmd in checks.items():
        out = run_cmd(cli, cmd)
        print(f"▶ {name}:")
        print(out if out else "❌ 기록 없음")
        print()

    cli.close()


def log_menu():
    while True:
        print("1. SSH_LOG")
        print("2. FTP_LOG")
        print("3. JUPYTER_LOG")
        print("0. 뒤로가기")
        sel = input("선택: ").strip()
        if sel == "1":
            ssh_log()
        elif sel == "2":
            ftp_log()
        elif sel == "3":
            jupyter_log()
        elif sel == "0":
            break

def menu_iscsi_server():
    while True:
        print("\n[iSCSI 서버 메뉴]")
        print("1. 서버 방화벽/SELinux 해제 + 설치")
        print("2. 타깃 구성")
        print("3. 서비스 재시작/포트 확인")
        print("0. 뒤로가기")
        sel = input("선택: ").strip()
        if sel == "1":
            iscsi_server_setup()
        elif sel == "2":
            iscsi_server_target_config()
        elif sel == "3":
            iscsi_server_restart()
        elif sel == "0":
            break

def menu_iscsi_client_samba_server():
    while True:
        print("\n<iSCSI 클라이언트 + Samba 서버 메뉴>")
        print("1. 클라이언트 설치 & IQN 생성/추출")
        print("2. iSCSI 로그인")
        print("3. iSCSI 마운트")
        print("4. 쿼터 설정(usrquota 자동)")
        print("5. Samba 서버 설정")
        print("6. NFS 서버 설정")
        print("0. 뒤로가기")
        sel = input("선택: ").strip()
        if sel == "1":
            iscsi_client_install()
            iscsi_client_iqn()
        elif sel == "2":
            iscsi_client_login()
        elif sel == "3":
            iscsi_client_mount()
        elif sel == "4":
            quota()
        elif sel == "5":
            samba_server_setup()
        elif sel == "6":
            nfs_server_setup()
        elif sel == "0":
            break
def menu_samba_client_os():
    while True:
        print("\n<Samba 클라이언트 + 운영 서버 메뉴>")
        print("1. 운영서버 기본 도구/방화벽 설정")
        print("2. Samba homes 마운트(서버=ISCSI_CLIENT_IP)")
        print("3. 유저 생성(웹용 2명)")
        print("4. Apache 설치")
        print("5. VirtualHost 설정(ww1/ww2)")
        print("6. FTP 설치/설정")
        print("7. PHP 설치/테스트 페이지")
        print("8. MariaDB 설치/초기화")
        print("9. phpMyAdmin 설치/설정")
        print("10. log메뉴 들어가기")
        print("0. 뒤로가기")
        sel = input("선택: ").strip()
        if sel == "1":
            tools_ufw()
        elif sel == "2":
            samba_mount()
        elif sel == "3":
            user_create()
        elif sel == "4":
            apache()
        elif sel == "5":
            virtualhost()
        elif sel == "6":
            ftp()
        elif sel == "7":
            php()
        elif sel == "8":
            mariadb()
        elif sel == "9":
            phpmyadmin()
        elif sel == "10":
            log_menu()    
        elif sel == "0":
            break
def menu_nfs_client():
    while True:
        print("\n<NFS 클라이언트 메뉴>")
        print("1. NFS 클라이언트 설정 (10.4.32.100)")
        print("0. 뒤로가기")
        sel = input("선택: ").strip()
        if sel == "1":
            nfs_client_mount()
        elif sel == "0":
            break
def dns_menu():
    while True:
        print("1. DNS 초기화")
        print("2. DNS 정보 추가")
        print("3. DNS 존 파일 출력")
        print("4. DNS 입력 저장")
        print("5. zone clear")
        print("0. 뒤로가기")
        choice = input("번호를 선택하세요: ")

        if choice == "1":
            dns_reset()
        elif choice == "2":
            add_dns()
        elif choice == "3":
            print_dns()
        elif choice == "4":
            data = dns_input()
            dns_save(data)
        elif choice == "5":
            clear_zone()
        elif choice == "0":
            print("프로그램을 종료합니다.")
            break
        else:
            print("잘못된 입력입니다. 다시 입력해주세요.\n")
def server():
    while True:
        print("\n<Server 메뉴>")
        print("1. iSCSI 서버")
        print("2. iSCSI 클라이언트 + Samba 서버 + NFS 서버")
        print("3. Samba 클라이언트 + 운영 서버")
        print("4. NFS 클라이언트")
        print("5. DNS 설정")
        print("0. 종료")
        sel = input("선택: ").strip()
        if sel == "1":
            menu_iscsi_server()
        elif sel == "2":
            menu_iscsi_client_samba_server()
        elif sel == "3":
            menu_samba_client_os()
        elif sel == "4":
            menu_nfs_client()
        elif sel == "5":
            dns_menu()
        elif sel == "0":
            print("프로그램 종료")
            break        
def network():
    while True:
        print("\n<Network 메뉴>")
        print("1. 실행 함수")
        print("0. 뒤로가기")
        sel = input("선택: ").strip()
        if sel == "1":
            run_cisco_command()
        elif sel == "0":
            print("프로그램 종료")
            break
def server_check_all():
    print("\n==============================")
    print("✅ 전체 서버 점검 시작")
    print("==============================")

    # 이미 너가 만든 점검 함수들
    check_iscsi_server()
    check_storage_server()
    check_web_server()
    check_nfs_client()

    # 로그 점검 추가
    check_storage_logs()
    check_web_logs()

    print("\n✅ 전체 서버 점검 완료")

def net_server():
    while True:
        print("\n<Server / Network>")
        print("1. Network")
        print("2. Server")
        print("3. 점검")   
        print("0. 종료")
        sel = input("선택: ").strip()

        if sel == "1":
            network()
        elif sel == "2":
            server()
        elif sel == "3":
            server_check_all()   
        elif sel == "0":
            print("프로그램 종료")
            break

net_server()
log_cli.close()

In [1]:
from netmiko import ConnectHandler

# === 1️⃣ 장비 정보 (기본 아이디/비밀번호 포함) ===
DEVICE_INFO = {
    "R1": {"ip": "10.4.0.1", "type": "cisco_ios", "username": "R1", "password": "cisco"},
    "R2": {"ip": "10.4.32.1", "type": "cisco_ios", "username": "R2", "password": "cisco"},
    "R3": {"ip": "10.4.64.1", "type": "cisco_ios", "username": "R3", "password": "cisco"},
    "R4": {"ip": "192.168.100.1", "type": "cisco_ios", "username": "R4", "password": "cisco"},
    "R5": {"ip": "10.4.128.1", "type": "cisco_ios", "username": "R5", "password": "cisco"},
    "R6": {"ip": "10.4.72.1", "type": "cisco_ios", "username": "R6", "password": "cisco"},
    "SW1": {"ip": "10.4.32.2", "type": "cisco_ios", "username": "S1", "password": "cisco"},
    "SW2": {"ip": "10.4.72.2", "type": "cisco_ios", "username": "S2", "password": "cisco"},
    "SW3": {"ip": "192.168.100.2", "type": "cisco_ios", "username": "S3", "password": "cisco"},
    "SW4": {"ip": "10.4.160.2", "type": "cisco_ios", "username": "S4", "password": "cisco"},
    "SW5": {"ip": "10.4.160.3", "type": "cisco_ios", "username": "S5", "password": "cisco"},
    "SW6": {"ip": "10.4.160.4", "type": "cisco_ios", "username": "S6", "password": "cisco"},
    "SW7": {"ip": "10.4.128.2", "type": "cisco_ios", "username": "S7", "password": "cisco"},
    "SW8": {"ip": "10.4.128.3", "type": "cisco_ios", "username": "S8", "password": "cisco"},
    "SW9": {"ip": "10.4.128.4", "type": "cisco_ios", "username": "S9", "password": "cisco"},
    "SW10": {"ip": "10.4.80.2", "type": "cisco_ios", "username": "S10", "password": "cisco"},
}

# === 2️⃣ 장비 선택 및 아이디/비밀번호 입력 ===
def select_device():
    print("접속할 장비 이름을 선택하세요:")
    print(f"사용 가능한 장비: {list(DEVICE_INFO.keys())}")
    
    choice = input("장비 이름 입력 (예: R1, SW1): ").strip().upper()

    if choice in DEVICE_INFO:
        info = DEVICE_INFO[choice].copy()  # 원본 보호 위해 복사
        info["name"] = choice

        # 사용자에게 아이디, 비밀번호 입력받기 (빈칸 시 기본값 유지)
        user = input("아이디를 입력하세요 : ".format(info["username"])).strip()
        pw = input("비밀번호를 입력하세요 : ".format(info["password"])).strip()

        if user:
            info["username"] = user
        if pw:
            info["password"] = pw

        return info
    else:
        print("❌ 잘못된 장비 이름입니다.")
        return None

# === 3️⃣ 명령 선택 ===
def select_command():
    commands = {
        1: "show running-config",
        2: "show ip interface brief",
        3: "show vtp status",
        4: "show vlan-switch brief",
        5: "show spanning-tree brief",
        6: "show mac address-table",
        7: "show frame-relay map",
        8: "show version",
        9: "show interfaces trunk",
        10: "show ip route",
        11: "ping 8.8.8.8",
    }

    print("==== Cisco Command Menu ====")
    for key, val in commands.items():
        print(f"{key}. {val}")
    print("=============================")

    try:
        choice = int(input("실행할 명령 번호를 입력하세요: "))
        if choice in commands:
            return commands[choice]
        else:
            print("❌ 잘못된 번호입니다.")
            return None
    except ValueError:
        print("❌ 숫자를 입력하세요.")
        return None

# === 4️⃣ 장비 접속 (SSH) ===
def connect_device(device_info):
    device = {
        "device_type": device_info["type"],
        "host": device_info["ip"],
        "username": device_info["username"],
        "password": device_info["password"],
        "secret": "cisco",  # enable password
        "fast_cli": False,
    }
    net_connect = ConnectHandler(**device)
    net_connect.enable()
    return net_connect

# === 5️⃣ 설정 저장 ===
def save_running_config(output, device_name, device_ip):
    file_name = f"{device_name}_{device_ip}_running-config.txt"
    with open(file_name, "w") as f:
        f.write(output)
    print(f"💾 전체 설정 백업 저장됨: {file_name}")

# === 6️⃣ 인터페이스 목록 추출 ===
def extract_interfaces(output):
    lines = output.splitlines()
    interface_lines = [line for line in lines if line.startswith("interface")]
    return [line.split()[1] for line in interface_lines if len(line.split()) > 1]

# === 7️⃣ IP route 추출 ===
def extract_ip_routes(output):
    lines = output.splitlines()
    return [line for line in lines if line.startswith("ip route")]

# === 8️⃣ 인터페이스 설정 출력 ===
def show_interface_config(output, interface_name):
    lines = output.splitlines()
    iface_config = []
    capture = False
    for line in lines:
        if line.startswith("interface "):
            capture = (line.split()[1] == interface_name)
        elif line.startswith("interface") and capture:
            capture = False
        
        if capture or line.startswith(f"interface {interface_name}"):
            if iface_config and iface_config[-1] == line:
                continue
            iface_config.append(line)
            
    print(f"\n--------- [{interface_name} 설정] ---------")
    if iface_config:
        if not iface_config[0].startswith("interface"): 
            iface_config = [f"interface {interface_name}"] + iface_config
        
        clean_config = [line.strip() for line in iface_config if line.strip()]
        
        print("\n".join(clean_config))
    else:
        print("(설정 없음)")
    print("----------------------------------------")

# === 9️⃣ 실행 함수 ===
def run_cisco_command():
    device_info = select_device()
    if not device_info:
        return

    cmd = select_command()
    if not cmd:
        return

    print(f"\n✅ 장비: {device_info['name']} ({device_info['ip']}) 실행할 명령: {cmd}\n")
    try:
        net_connect = connect_device(device_info)
        output = net_connect.send_command(cmd)
        net_connect.disconnect()

        if "running-config" in cmd:
            save_running_config(output, device_info["name"], device_info["ip"])
            interfaces = extract_interfaces(output)
            ip_routes = extract_ip_routes(output)

            if interfaces:
                print("--------- [인터페이스 목록] ---------")
                for idx, iface in enumerate(interfaces, 1):
                    print(f"{idx}. {iface}")
                print("-------------------------------------")
                try:
                    iface_choice = int(input("확인할 인터페이스 번호를 입력하세요: "))
                    if 1 <= iface_choice <= len(interfaces):
                        show_interface_config(output, interfaces[iface_choice - 1])
                except ValueError:
                    print("❌ 잘못된 입력입니다.")
                except IndexError:
                    print("❌ 잘못된 번호입니다.")
            else:
                print("(인터페이스 설정 없음)")

            print("--------- [IP Route 설정] ---------")
            if ip_routes:
                print("\n".join(ip_routes))
            else:
                print("(설정 없음)")
            print("----------------------------------")

        else:
            print("--------- [명령 결과] ---------")
            print(output)
            print("--------------------------------")

    except Exception as e:
        print(f"❌ 연결 또는 명령 실행 실패: {e}")

# === 10️⃣ 메인 실행 ===
if __name__ == "__main__":
    run_cisco_command()


ModuleNotFoundError: No module named 'netmiko'